In [1]:
import sys
sys.path.append('C://Users//malko/anaconda3/envs/py38_64/lib/site-packages/dart-fss/')
# sys.path

In [3]:
from dart_fss import api, auth, corp, errors, filings, fs, utils, xbrl
from dart_fss.auth import set_api_key, get_api_key
from dart_fss.corp import get_corp_list
from dart_fss.filings import search
from dart_fss.fs import extract
from dart_fss.xbrl import get_xbrl_from_file

In [4]:
api_key = open('crtfc_key.txt',mode='r').readline()
set_api_key(api_key=api_key)
print("api set 완료")

api set 완료


In [5]:
def stock_code_reset(stock_code):
    corp_code = corp_df[corp_df['stock_code'] == stock_code].iloc[0, 0]
    return corp_code

import pandas as pd
import xml.etree.ElementTree as ET

tree = ET.parse('Raw/Corpcode/corp_num/CORPCODE.xml')
root = tree.getroot()

corp_df = pd.DataFrame(columns=['corp_code','corp_name','stock_code','modify_date'])
for company in root.iter('list'):
    stock_code = company.findtext('stock_code')
    stock_code = stock_code.strip()
    if stock_code:
        company_dict = {
            'corp_code':company.findtext('corp_code'),
            'corp_name':company.findtext('corp_name'),
            'stock_code':company.findtext('stock_code'),
            'modify_date':company.findtext('modify_date')
        }
        corp_df = corp_df.append(company_dict, ignore_index=True)
        
import warnings
warnings.filterwarnings('ignore') # 경고메시지 무시하기

import FinanceDataReader as fdr
# stock_list_kospi = fdr.StockListing('KOSPI')['Symbol'].to_list()
# stock_list_kosdaq = fdr.StockListing('KOSDAQ')['Symbol'].to_list()
stock_df_del = fdr.StockListing('KRX-DELISTING')
stock_list_del = stock_df_del[(stock_df_del['Market'] == "KOSPI") | (stock_df_del['Market'] == "KOSDAQ")]['Symbol'].to_list()

stock_list = stock_list_del

import re
# stock_code_list 정제하기 (우선주, 리츠, 옵션) -> 6자리 숫자만, 맨 앞자리에 0 들어가는 것만
stock_code_list = []
for stock_code in stock_list:
    stock_code = re.findall("\d+", stock_code)[0] # 숫자만 추출한 뒤 첫번째 것만 사용
    if (stock_code[-1] == '0') and (len(stock_code) == 6):
        stock_code_list.append(stock_code)
len(stock_code_list)

# corp_list = get_corp_list()
# none_list=[]
# for stock_code in stock_code_list:
#     data = corp_list.find_by_stock_code(stock_code)
#     if data == None:
#         none_list.append(stock_code)
        
# def f(title):
#     return title[:6]

# import os
# data_path = './Raw/fssdata_kosdaq/'
# files_list_kosdaq1 = os.listdir(data_path)
# data_path = './Raw/fssdata_temp1/'
# files_list_kosdaq2 = os.listdir(data_path)
# data_path = './Raw/fssdata_temp2/'
# files_list_kosdaq3 = os.listdir(data_path)
# data_path = './Raw/fssdata_temp3/'
# files_list_kosdaq4 = os.listdir(data_path)
# data_path = './Raw/fssdata_temp4/'
# files_list_kosdaq5 = os.listdir(data_path)
# files_list_kosdaq1 = list(map(f,files_list_kosdaq1))
# files_list_kosdaq2 = list(map(f,files_list_kosdaq2))
# files_list_kosdaq3 = list(map(f,files_list_kosdaq3))
# files_list_kosdaq4 = list(map(f,files_list_kosdaq4))
# files_list_kosdaq5 = list(map(f,files_list_kosdaq5))

# data_path = './Raw/KOSDAQ by dartfss/'
# files_list_kosdaq = os.listdir(data_path)
# files_list_kosdaq = list(map(f,files_list_kosdaq))

# for stock in none_list:
#     try:
#         stock_code_list.remove(stock)
#     except:
#         continue
#         print(stock)

len(stock_code_list)

1406

In [ ]:
corp_list = get_corp_list()
none_list=[]
for stock_code in stock_code_list:
    data = corp_list.find_by_stock_code(stock_code)
    if data == None:
        none_list.append(stock_code)

In [ ]:
len(stock_code_list)

0

In [6]:
import numpy as np
stock_code_list_reset = []
for stock_code in stock_code_list:
    if not np.all(corp_df[corp_df['stock_code'] == stock_code] == None):
        stock_code_list_reset.append(stock_code)
len(stock_code_list_reset)

831

In [9]:
for stock in files_list_kosdaq:
    try:
        stock_code_list_reset.remove(stock)
    except:
        continue
        print(stock)

In [10]:
len(stock_code_list_reset)

160

In [7]:
error_df = pd.read_csv('./Raw/error_df.csv',index_col=0)
stock_code_list = [str('%06d'%(int(stock_code))) for stock_code in error_df.kospi_stock]
# len(error_df)

In [5]:
error_list_rcp = []
stock_code = '016610'
fs,error_list_rcp = extract(corp_code = stock_code_reset(stock_code), bgn_de='20130101',end_de='20150625',report_tp='quarter',
              error_list=error_list_rcp)

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/5 [00:00<?, ?report/s]

In [8]:
error_list_rcp = []
error_list_stock = []
for stock_code in stock_code_list:
#     if stock_code_list_reset.index(stock_code) % 20 == 0:
#         print(stock_code_list_reset.index(stock_code))
    try:
        fs, error_list_rcp = extract(corp_code = stock_code_reset(stock_code), bgn_de='20130101',report_tp='quarter',
                    error_list = error_list_rcp)
        fs.save(filename = '{}.xlsx'.format(stock_code), path="Raw/KOSPI by dartfss/")
    except Exception as e:
        error_list_stock.append(stock_code)

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/7 [00:00<?, ?report/s]

...

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

In [9]:
error_list_rcp

['20140331002409',
 '20140814001137',
 '20131114001075',
 '20141114001111',
 '20130814001400',
 '20141114000698',
 '20210318001330',
 '20210318001350',
 '20210427000500',
 '20210427000450',
 '20210427000389',
 '20181023000277',
 '20160816001353',
 '20150817001489',
 '20210517001542',
 '20210427000513',
 '20210427000485',
 '20210427000463',
 '20210427000437',
 '20210427000402',
 '20210427000369',
 '20181023000281',
 '20181023000269',
 '20161114001858',
 '20160513004804',
 '20151116001425',
 '20150515001745',
 '20200320001044',
 '20171117000441',
 '20171117000438',
 '20171117000482',
 '20160816001620',
 '20140814000271',
 '20161110000255',
 '20160516002774',
 '20141114000337',
 '20140515001564',
 '20210318001055',
 '20190322001298',
 '20180323001120',
 '20190814002895',
 '20210621000217',
 '20190621000460',
 '20150515002680',
 '20140814000944',
 '20210317000982',
 '20200330002776',
 '20190710000341',
 '20180402004121',
 '20170424000672',
 '20160415000633',
 '20150415000389',
 '2014042500

In [11]:
error_list_rcp_kosdaq = []
error_list_stock_kosdaq = []
for stock_code in stock_code_list_reset:
    if stock_code_list_reset.index(stock_code) % 20 == 0:
        print(stock_code_list_reset.index(stock_code))
    try:
        fs, error_list_rcp_kosdaq = extract(corp_code = stock_code_reset(stock_code), bgn_de='20130101',report_tp='quarter',
                    error_list = error_list_rcp_kosdaq)
        fs.save(filename = '{}.xlsx'.format(stock_code), path="Raw/KOSDAQ by dartfss")
    except Exception as e:
        error_list_stock_kosdaq.append(stock_code)

0


Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/8 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

...

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/8 [00:00<?, ?report/s]

In [10]:
# error_list_rcp_del = []
# error_list_stock_del = []
for stock_code in stock_code_list_reset[200:]:
    if stock_code_list_reset.index(stock_code) % 20 == 0:
        print(stock_code_list_reset.index(stock_code))
    try:
        fs, error_list_rcp_del = extract(corp_code = stock_code_reset(stock_code), bgn_de='20130101',report_tp='quarter',
                    error_list = error_list_rcp_del)
        fs.save(filename = '{}.xlsx'.format(stock_code), path="Raw/DEL by dartfss/")
    except Exception as e:
        error_list_stock_del.append(stock_code)

200


Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/7 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

...

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/1 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/1 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

In [8]:
for stock_code in stock_code_list_reset[100:200]:
    if stock_code_list_reset.index(stock_code) % 20 == 0:
        print(stock_code_list_reset.index(stock_code))
    try:
        fs, error_list_rcp_del = extract(corp_code = stock_code_reset(stock_code), bgn_de='20130101',report_tp='quarter',
                    error_list = error_list_rcp_del)
        fs.save(filename = '{}.xlsx'.format(stock_code), path="Raw/DEL by dartfss/")
    except Exception as e:
        error_list_stock_del.append(stock_code)

100


Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/5 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/2 [00:00<?, ?report/s]

...

Semiannual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/7 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/4 [00:00<?, ?report/s]

In [22]:
error_list_rcp = []
error_list_stock = []
for stock_code in stock_code_list_reset[8:]:
    if stock_code_list_reset.index(stock_code) % 20 == 0:
        print(stock_code_list_reset.index(stock_code))
    try:
        fs = extract(corp_code = stock_code_reset(stock_code), bgn_de='20130101',report_tp='quarter')
        fs.save(filename = '{}.xlsx'.format(stock_code), path="Raw/KOSPI by dartfss/")
    except Exception as e:
        error_list_stock.append(stock_code)

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

...

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/9 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/8 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/17 [00:00<?, ?report/s]

In [11]:
dic = {'kospi_stock':error_list_stock_del,'kospi_rcp':error_list_rcp_del}
error_df = pd.DataFrame.from_dict(dic, orient='index')
error_df = error_df.transpose()
error_df
error_df.to_csv("Raw/error_del.csv")

In [ ]:
len(error_list_stock)

44

In [20]:
a = 1
test(2)

(2, 2)

In [15]:
a

3

In [19]:
def test(b):
    global a
    a =a+1
#     k.append(b)
    return a,b

In [128]:
error_list_rcp

[]

In [110]:
error_list = ['20171114001802','20170331004936','20160816001586','20161114001577','20160512003249','20170823000032',
             '20180412001123','20171110000252','20171103000070','20130214000297','20181108000387','20201109000188',
             '20200814000003','20170103000377','20180814002995','20141114000465','20200814000028','20160516001712']